In [1]:
import pandas as pd
import os

In [2]:
out_clones_HL = "AMC-TC-x001-HL-clones.csv"

In [3]:
myfiles = [x for x in os.listdir() if x.endswith("-allinfo-filtered.csv")]
myfiles

['AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered.csv']

In [4]:
def readFile(myfile):
    sample = myfile.split("_")[0]
    celltype = myfile.split("-")[-3]
    df = pd.read_csv(myfile, sep="\t")
    df["Sample"] = sample
    df["CellType"] = celltype
    df["CellID"] = [x.split("_")[0] for x in df["acc"]]
    return(df)

In [5]:
df = readFile(myfiles[0])
for myfile in myfiles[1:]:
    df_tmp = readFile(myfile)
    df = pd.concat([df, df_tmp])
df.tail()

,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3pepshort,cdr3nucshort,cdr3_qual_min,...,nr_v_subs,nr_v_alleles,nr_j_subs,nr_j_alleles,acc:2,readingframe:1,nr_sites,Sample,CellType,CellID
1425,TTTGGTTGTGGTGTAG-1_contig_2,nomatch,nomatch,nomatch,0,CGTWDSSLSVWVFG,TGCGGAACATGGGATAGCAGCCTGAGTGTGTGGGTGTTCGGC,CGTWDSS,TGCGGAACATGGGATAGCAGC,60,...,1,1,1,1,NaN,NaN,NaN,AMC-TC-x001-AK1286,IGL_HUMAN,TTTGGTTGTGGTGTAG-1
1426,TTTGTCAGTTTGCATG-1_contig_3,nomatch,nomatch,nomatch,0,CSSYTSSSLVFG,TGCAGCTCATATACAAGCAGCAGCCTGGTGTTCGGC,CSSYT,TGCAGCTCATATACA,60,...,1,1,1,1,NaN,NaN,NaN,AMC-TC-x001-AK1286,IGL_HUMAN,TTTGTCAGTTTGCATG-1
1427,TTTGTCATCCGCGCAA-1_contig_2,nomatch,nomatch,nomatch,0,CGTWDSSLSAGVFG,TGCGGAACATGGGATAGCAGCCTGAGTGCTGGGGTGTTCGGC,CGTWDSS,TGCGGAACATGGGATAGCAGC,60,...,1,1,1,1,NaN,NaN,NaN,AMC-TC-x001-AK1286,IGL_HUMAN,TTTGTCATCCGCGCAA-1
1428,TTTGTCATCCGCGGTA-1_contig_1,nomatch,nomatch,nomatch,1,CQSADSSNAYVFG,TGTCAATCAGCAGACAGTAGTAATGCTTATGTCTTCGGA,CQSADS,TGTCAATCAGCAGACAGT,40,...,1,1,1,1,NaN,NaN,NaN,AMC-TC-x001-AK1286,IGL_HUMAN,TTTGTCATCCGCGGTA-1
1429,TTTGTCATCGTGGACC-1_contig_1,nomatch,nomatch,nomatch,2,CAAWDDSLNAPVFG,TGTGCAGCATGGGATGACAGCCTGAATGCCCCGGTGTTCGGC,CAAWDDS,TGTGCAGCATGGGATGACAGC,60,...,1,1,1,1,NaN,NaN,NaN,AMC-TC-x001-AK1286,IGL_HUMAN,TTTGTCATCGTGGACC-1


In [6]:
df_heavy = df[df["CellType"] == "IGH_HUMAN"]
df_light = df[df["CellType"] != "IGH_HUMAN"]
print("Heavy:", len(df_heavy))
print("Light:", len(df_light))

Heavy: 15765
Light: 18130


In [7]:
cols = ["CellID","CellType","cdr3pep"]
df_heavy = df_heavy[cols]
df_light = df_light[cols]
df_HL = pd.merge(df_heavy, df_light, how="inner", on="CellID")
df_HL['freq'] = 1
df_HL.head()

,CellID,CellType_x,cdr3pep_x,CellType_y,cdr3pep_y,freq
0,AAACCTGAGGCTAGGT-1,IGH_HUMAN,CAKDHSSWYEWYSLAPGDYWGQGTLVT,IGK_HUMAN,CQQYGSSPLFTFG,1
1,AAACCTGAGGCTAGGT-1,IGH_HUMAN,CAKDHSSWYEWYSLAPGDYWGQGTLVT,IGK_HUMAN,CQQYGSSPLFTFG,1
2,AAACCTGAGGCTAGGT-1,IGH_HUMAN,CAKDHSSWYEWYSLAPGDYWGQGTLVT,IGK_HUMAN,CQQYGSSPLFTFG,1
3,AAACCTGAGGCTAGGT-1,IGH_HUMAN,CAKDHSSWYEWYSLAPGDYWGQGTLVT,IGK_HUMAN,CQQYGSSPLFTFG,1
4,AAACCTGAGTGCGATG-1,IGH_HUMAN,CATHLIGSHFDYWGQGTLVT,IGK_HUMAN,CQQYDSYSTFG,1


In [8]:
# Create short version of the CDR3's
df_HL["cdr3pep_heavy"] = [x[:-7] for x in df_HL["cdr3pep_x"]]
df_HL["cdr3pep_light"] = [x[:-1] for x in df_HL["cdr3pep_y"]]

In [9]:
df_HL_clones = df_HL.groupby(["CellID", "CellType_x", "cdr3pep_heavy", "CellType_y", "cdr3pep_light"]).agg(sum)
df_HL_clones = df_HL_clones.reset_index()
df_HL_clones = df_HL_clones.sort_values(by="freq", ascending=False)

In [10]:
df_HL_clones.head(50)

,CellID,CellType_x,cdr3pep_heavy,CellType_y,cdr3pep_light,freq
2964,GGATTACCATCTACGA-1,IGH_HUMAN,CARWDYDFWSGYYTGRASSGANFDYW,IGK_HUMAN,CLQHNSYPRTF,32
4239,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQYNSYPYTF,32
4238,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQYNNWLTWTF,32
4237,TGGCGCAAGTAGCCGA-1,IGH_HUMAN,CARGGGDSSGFSPGRIDSW,IGK_HUMAN,CQQRSNWPPFTF,32
4399,TTCCCAGGTAAACCTC-1,IGH_HUMAN,CASKDLSRGFDYW,IGK_HUMAN,CQQRSNWPPRLTF,28
1806,CGGACTGGTATGCTTG-1,IGH_HUMAN,CARGGHGHDYGDFLPWGDQLGGFRYYYYGMDVW,IGK_HUMAN,CQQYNSYSPMYTF,24
2776,GCGCCAACACTTAACG-1,IGH_HUMAN,CAKDLTVHLTRYFYYYYGMDVW,IGK_HUMAN,CHQSGSPGFTF,24
4114,TGCACCTGTCTAAACC-1,IGH_HUMAN,CARDDGTLALDYW,IGL_HUMAN,CCSYAGSSTSPYVF,24
1807,CGGACTGGTATGCTTG-1,IGH_HUMAN,CARGGHGHDYGDFLPWGDQLGGFRYYYYGMDVW,IGL_HUMAN,CGTWDSSLSAYVF,24
4395,TTCCCAGCATTTCAGG-1,IGH_HUMAN,CARDSPGEGFCW,IGK_HUMAN,CQQRSNWPPRYTF,20


In [11]:
df_HL_clones.to_csv(out_clones_HL)
print("Wrote", out_clones_HL, "to disk")

Wrote AMC-TC-x001-HL-clones.csv to disk
